# Interpreting the Bible with Claude

In [1]:
# Importing the libraries
import os
import anthropic
from anthropic import Anthropic
import pandas as pd
import json

# Setting the working directory
def find_repo_root(repo_name='orare-model'):
    current_dir = os.getcwd()
    while current_dir != '/':
        if os.path.basename(current_dir) == repo_name:
            return current_dir
        current_dir = os.path.dirname(current_dir)
    raise FileNotFoundError(f"Repository root '{repo_name}' not found.")

repo_root = find_repo_root()
os.chdir(repo_root)


In [2]:
# Loading the data
with open('bible/data/cat_bible_preproc.json') as f:
    data = json.load(f)

# Converting json into df
bible_data = pd.DataFrame(data)
bible_data.head()

,pasaje,texto,texto_cat,area_vida
0,1 Corintios 10:12,"Así que, el que piensa estar firme, mire no ca...","Por eso, el que se cree muy seguro, ¡cuídese d...",[orgullo]
1,1 Corintios 10:13,"No os ha tomado tentación, sino humana: mas fi...","Hasta ahora, ustedes no tuvieron tentaciones q...","[crecimiento espiritual, maternidad, vicio]"
2,1 Corintios 10:31,"Si pues coméis, ó bebéis, ó hacéis otra cosa, ...","En resumen, sea que ustedes coman, sea que beb...",[descanso y vacaciones]
3,1 Corintios 11:9,Porque tampoco el varón fué criado por causa d...,"ni fue creado el hombre a causa de la mujer, s...",[creación/evolución]
4,1 Corintios 13:13,"Y ahora permanecen la fe, la esperanza, y la c...","En una palabra, ahora existen tres cosas: la f...",[amor]


In [3]:
bible_data['texto'] = bible_data['texto_cat']
bible_data.drop(columns=['texto_cat'], inplace=True)
bible_data = bible_data[['pasaje','texto','area_vida']]
bible_data.head()

,pasaje,texto,area_vida
0,1 Corintios 10:12,"Por eso, el que se cree muy seguro, ¡cuídese d...",[orgullo]
1,1 Corintios 10:13,"Hasta ahora, ustedes no tuvieron tentaciones q...","[crecimiento espiritual, maternidad, vicio]"
2,1 Corintios 10:31,"En resumen, sea que ustedes coman, sea que beb...",[descanso y vacaciones]
3,1 Corintios 11:9,"ni fue creado el hombre a causa de la mujer, s...",[creación/evolución]
4,1 Corintios 13:13,"En una palabra, ahora existen tres cosas: la f...",[amor]


In [4]:
# Initializing the Anthropic client
client = Anthropic(
    api_key=os.environ.get("ANTHROPIC_API_KEY")
)

In [5]:
# Defining the system message
sys_message = "Eres un experto en interpretación bíblica con un profundo entendimiento de la vida moderna. Tu tarea es interpretar pasajes de la Biblia de una manera que sea relevante y aplicable a situaciones cotidianas actuales."

In [6]:
# Defining the prompt instructions
instructions1 = """
# Tarea #
Para cada pasaje bíblico proporcionado:
1. Lee cuidadosamente el texto.
2. Identifica el mensaje principal o la enseñanza clave.
3. Crea una situación moderna que refleje el tema central del pasaje.
4. Describe esa situación moderna y explica como el mensaje del pasaje se aplica.
- **No incluyas etiquetas como '# Output #' en tu respuesta.**
- **No incluyas saltos de línea en tu respuesta.**
- **No incluyas etiquetas como "Situación moderna:" o "Explicación:"**

Tu Respuesta debe tener lo siguiente:
Crea una situación moderna que refleje el tema central del pasaje. Describe esa situación moderna y explica como el mensaje del pasaje se aplica.
# fin Tareas #

# Estructura input #
id_libro|fragmento
pasaje1|texto1\n
pasaje2|texto2\n
pasajen|texton
# Fin estructura input #

# Input #"""

instructions2 = """# Fin Input #

# Estructura Output #
id_libro|respuesta
pasaje1|respuesta1\n
pasaje2|respuesta2\n
pasajen|respuestan
# Fin Estructura Output #
"""

In [7]:
output_list = []
input_tokens_list = []
output_tokens_list = []
# Process to get the interpretion of all the Bible in batches of 10 rows
for i in range(0, len(bible_data), 10):
    # Getting the batch
    batch = bible_data.iloc[i:i+10]
    # Creating the input prompt
    input_data = "\n".join([f"{row['pasaje']}|{row['texto']}" for _, row in batch.iterrows()])
    prompt = f"""{instructions1}{input_data}{instructions2}"""
    # Getting the output
    output_data = client.messages.create(
                            model="claude-3-5-sonnet-20240620",
                            max_tokens=8192,
                            temperature=0,
                            system=sys_message,
                            messages=[
                                {
                                    "role": "user",
                                    "content": [
                                        {
                                            "type": "text",
                                            "text": prompt
                                        }
                                    ]
                                }
                            ]
                        )
    # Splitting the output
    output_data_splitted = output_data.content[0].text.split("\n\n")
    # List to count the amount of tokens used
    input_tokens_list.append(output_data.usage.input_tokens)
    output_tokens_list.append(output_data.usage.output_tokens)
    # Creating the list with the output content
    for response in output_data_splitted:
        output_list.append(response)

In [11]:
# Calculating the estimated costs
input_cost = 3/1000000
output_cost = 15/1000000
total_cost = sum(input_tokens_list)*input_cost + sum(output_tokens_list)*output_cost
total_cost

print(f"Input tokens: {sum(input_tokens_list)}")
print(f"Output tokens: {sum(output_tokens_list)}")
print(f"Total cost: {total_cost}")


Input tokens: 84269
Output tokens: 112842
Total cost: 1.945437


In [26]:
# Splitting the output into pasaje and interpretacion
list_pasaje = []
list_interpretacion = []
for i in output_list:
    pasaje, interpretacion = i.split("|")
    list_pasaje.append(pasaje)
    list_interpretacion.append(interpretacion)

In [53]:
# Creating the final df
bible_data['interpretacion'] = list_interpretacion
bible_data['id'] = [i for i in range(1, len(bible_data)+1)]
bible_data = bible_data[['id','pasaje','texto','interpretacion','area_vida']]

In [57]:
# Convert DataFrame to a dictionary
bible_data_json = bible_data.to_dict(orient='records')

# Save the list as a JSON file
with open('bible/data/cat_bible_int_claude.json', 'w', encoding='utf-8') as json_file:
    json.dump(bible_data_json, json_file, ensure_ascii=False, indent=4)